In [127]:
def stock_path(S0, level, M=4):
    h = T/np.power(M,level)
    dW = np.random.normal(size = np.power(M, level)//T)
    S = [S0]
    for i in range(np.power(M, level)//T):
        S_i = S[-1]+ a*(b-S[-1])*h + sigma*np.sqrt(S[-1])*dW[i]
        S.append(S_i)
    return S
T=1
a, b, sigma, S0, K = 0.15, 0.2, 0.3, 5, 4
print(stock_path(S0, level=3, M=4))
a, b = 0.15, 0.2

[5, 4.699130162573031, 4.431122948918697, 3.497720201842458, 3.3760631661599327, 3.755375476203618, 3.419573954378954, 2.796886513906659, 2.777235501717933, 3.1580556221493934, 2.4704439074430713, 2.3725425322529956, 2.6705806178212033, 3.2145635435497653, 2.6937714113286177, 3.3697183174548515, 3.716114487769228, 4.621258674408149, 4.515040593054652, 4.402285620662201, 5.389344431435495, 5.697009334853544, 4.66204871675663, 3.9155971126639235, 3.754969555386724, 3.4249691871563663, 3.8224665112315166, 3.2298322405421405, 2.640385524690769, 2.2624823717317875, 1.8426018133379745, 1.4443325318730262, 1.9664606856028224, 2.0560834340992877, 2.1801443116134394, 2.6937068904717947, 1.5865452163924014, 1.9418206158323534, 1.1709414952882766, 1.2953298484464186, 1.7064609110312063, 1.4824346975104041, 1.6675510164381218, 1.096834282860865, 1.2457571181712295, 1.3046742612505289, 1.087472194377427, 0.9720325939937472, 0.35665234619928066, 0.6883114960907941, 0.6757809941610279, 0.296545340985

In [54]:
def payoff_call(S, K):
    return max(np.mean(S)-K, 0)
def payoff_put(S, K):
    return max(K-np.mean(S), 0)

In [137]:
import numpy as np

def mlmc(f, eps):
    """
    Multilevel Monte Carlo algorithm for computing E[f], where f is an expensive-to-evaluate function.
    
    Parameters:
    -----------
    f : function
        The function to be aproximated.
    L : int
        The number of levels of refinement.
    eps : float
        The desired accuracy of the estimator.
   
    
    Returns:
    --------
    payoffs : float
        The estimated value of f.
    """
    L = 0
    Nl = 10**4
    converged = False
    V_L_sum = []
    M=4
    
    while not converged:
        print(L)
        #print(1)
        if L<2:

            payoffs = []
            for _ in range(Nl):
                payoffs.append(f(stock_path(S0, level=L, M=4),K))

            Y_hat = np.mean(payoffs)
            V_L_hat = np.var(payoffs)
            V_L_sum.append(V_L_hat)

            Nl = int(2*eps**(-2)*(V_L_hat)**(0.5))
            
            Y_l_1 = Y_hat
            L+=1

        elif L>=2 and not converged:
            
            payoffs = []
            prod = 0
            for i in range(L):
                prod+=np.sqrt(V_L_sum[i]*T/np.power(M,i))

            Nl = int(2*eps**(-2)*np.sqrt(V_L_hat)*prod)

            for _ in range(Nl):
                payoffs.append(np.nan_to_num(f(stock_path(S0, level=L, M=4),K)))


            Y_l_1 = Y_hat    
            Y_hat = np.mean(payoffs[0:-1])
            V_L_hat = np.var(payoffs)
            
            V_L_sum.append(V_L_hat)

            if abs(Y_hat-Y_l_1)<1/np.sqrt(2)*(M**2-1)*eps:
                
                converged = True
                
            else:
                
                L+=1
        
        if L>15:
            return payoffs
            break
        
    return payoffs

In [138]:
res = mlmc(f=payoff_call, eps=0.05)

0
1
2


In [ ]:
print(np.mean(res))